# This notebook will be mainly used for the capstone project.

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

In [3]:
pip install beautifulsoup4

     |████████████████████████████████| 115 kB 4.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lxml

     |████████████████████████████████| 5.5 MB 4.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install geopy

     |████████████████████████████████| 111 kB 4.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install folium

     |████████████████████████████████| 93 kB 1.7 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


##  Scraping a site web and import the data into a pandas dataframe

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
import pgeocode
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

res  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table= soup.find_all('table')[0] 
df   = pd.read_html(str(table))
df   = pd.DataFrame(df[0])
df.shape

(180, 3)

### Now, the dataframe df2 contains the information without "Not assigned" rows

In [10]:
df2=df.query('Borough != "Not assigned" & Neighbourhood != "Not assigned"')
df2.shape

(103, 3)

## Add "Latitude" and "Longitude" to dataframe df2

In [11]:
df2["Latitude"]=""
df2["Longitude"]=""

for i in range(len(df2)):     
    nomi = pgeocode.Nominatim('ca')
    n=nomi.query_postal_code(df2.iloc[i, 0])        
    df2.iloc[i, 3]=n["latitude"]
    df2.iloc[i, 4]=n["longitude"]

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

## The postal code "M7R" is empty. That is why, The information was completed manually.

In [12]:
df2.loc[114,"Latitude"]="43.6370"
df2.loc[114,"Longitude"]="79.6158"

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
df2.loc[114,]

Postal Code                                        M7R
Borough                                    Mississauga
Neighbourhood    Canada Post Gateway Processing Centre
Latitude                                       43.6370
Longitude                                      79.6158
Name: 114, dtype: object

## The dataframe "df2" does not have empty values

In [14]:
df2.isnull().sum()

Postal Code      0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

In [15]:
df2.shape

(103, 5)

## Number of Boroughs and Neighborhoods

In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df2['Borough'].unique()),
        df2.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


## Create a map with geopy library

In [17]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [18]:
# create map of New York using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

In [19]:
toronto_data = df2[df2['Borough'] == 'Etobicoke'].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.6505,-79.5517
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.6437,-79.5767
3,M9P,Etobicoke,Westmount,43.6949,-79.5323
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.6898,-79.5582


### Let's get the geographical coordinates of Etobicoke.

In [20]:
address = 'Etobicoke, CA'

geolocator = Nominatim(user_agent="et_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of West Etobicoke City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of West Etobicoke City are 43.6435559, -79.5656326.


As we did with all of Etobicoke City, let's visualizate West Toronto the neighborhoods in it.

In [21]:
# create map of Manhattan using latitude and longitude values
map_et = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_et)  
    
map_et

## Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = 'Q4OWEJTPY4B4JHIKK3UX5BXNNCLKT4RLARFDRXSK053Z2XKP' # your Foursquare ID
CLIENT_SECRET = 'CBOBYZH13QPULLCDW0TDWGS0P1E33VLNDMGQTTZQOLNU05M1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q4OWEJTPY4B4JHIKK3UX5BXNNCLKT4RLARFDRXSK053Z2XKP
CLIENT_SECRET:CBOBYZH13QPULLCDW0TDWGS0P1E33VLNDMGQTTZQOLNU05M1


#### Let's explore the first neighborhood in our dataframe.

In [23]:
toronto_data.loc[0, 'Neighbourhood']

'Islington Avenue, Humber Valley Village'

In [24]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Islington Avenue, Humber Valley Village are 43.6662, -79.5282.


#### Now, let's get the top 100 venues that are in Toronto within a radius of 500 meters.

In [25]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=Q4OWEJTPY4B4JHIKK3UX5BXNNCLKT4RLARFDRXSK053Z2XKP&client_secret=CBOBYZH13QPULLCDW0TDWGS0P1E33VLNDMGQTTZQOLNU05M1&v=20180605&ll=43.6662,-79.5282&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb83125e6b2a16984d12e07'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Edenbridge - Humber Valley',
  'headerFullLocation': 'Edenbridge - Humber Valley, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.670700004500006,
    'lng': -79.52199076544632},
   'sw': {'lat': 43.6616999955, 'lng': -79.53440923455368}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d6d1a29cf7e41bd25ba8285',
       'name': 'TD Canada Trust',
       'location': {'address': '1498 Islington Ave',
        'lat': 43.6625453,
        'lng': -79.5317488,
        'labeledLatLngs': [{'label': 'display',
   

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,TD Canada Trust,Bank,43.662545,-79.531749
1,Shoppers Drug Mart,Pharmacy,43.663067,-79.531753
2,Humber Valley Village Baseball Diamond,Baseball Field,43.664369,-79.526997
3,Humber Valley Park,Park,43.664825,-79.524999
4,Humber Valley Rink,Skating Rink,43.664826,-79.524873


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=df2['Neighbourhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

#### Let's check the size of the resulting dataframe

In [32]:
print(toronto_venues.shape)
toronto_venues.head()

(2197, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.7276,-79.3148,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.7276,-79.3148,Portugril,43.725819,-79.312785,Portuguese Restaurant


### Let's check how many venues were returned for each neighborhood

In [33]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",6,6,6,6,6,6
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Berczy Park,90,90,90,90,90,90
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",38,38,38,38,38,38
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


#### Let's find out how many unique categories can be curated from all the returned venues

In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 265 uniques categories.


## Analyze Each Neighborhood

In [35]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### And let's examine the new dataframe size.

In [36]:
toronto_onehot.shape

(2197, 265)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bayview Village,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.0,0.000000,0.047619,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Berczy Park,0.011111,0.000000,0.000000,0.0,0.000000,0.011111,0.022222,0.000000,0.000000,...,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.026316,0.000000,0.0,0.000000,0.000000,0.026316,0.000000,0.026316,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.017857,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000


#### Let's confirm the new size

In [38]:
toronto_grouped.shape

(96, 265)

#### Let's print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Breakfast Spot   0.2
2            Badminton Court   0.2
3        Shanghai Restaurant   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
                   venue  freq
0  Performing Arts Venue  0.12
1                    Pub  0.12
2            Coffee Shop  0.12
3                    Gym  0.12
4               Pharmacy  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0  Middle Eastern Restaurant  0.17
1        Fried Chicken Joint  0.17
2              Deli / Bodega  0.17
3                Pizza Place  0.17
4                Coffee Shop  0.17


----Bayview Village----
                 venue  freq
0                 Park  0.25
1          Flower Shop  0.25
2          Gas Station  0.25
3                Trail  0.25
4  Moroccan Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwi

#### Let's put that into a _pandas_ dataframe

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shanghai Restaurant,Badminton Court,Breakfast Spot,Latin American Restaurant,Skating Rink,Women's Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
1,"Alderwood, Long Branch",Coffee Shop,Performing Arts Venue,Sandwich Place,Gym,Pub,Pizza Place,Pharmacy,Convenience Store,Cupcake Shop,Cuban Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Mediterranean Restaurant,Pizza Place,Middle Eastern Restaurant,Coffee Shop,Deli / Bodega,Fried Chicken Joint,Farmers Market,Escape Room,Ethiopian Restaurant,Event Space
3,Bayview Village,Flower Shop,Gas Station,Trail,Park,Event Space,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Café,Greek Restaurant,Sushi Restaurant,Pub,Liquor Store,Juice Bar,Thai Restaurant


## Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 5 clusters.

In [42]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 4, 1, 1, 1, 1, 1, 1], dtype=int32)

In [43]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shanghai Restaurant,Badminton Court,Breakfast Spot,Latin American Restaurant,Skating Rink,Women's Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
1,"Alderwood, Long Branch",Coffee Shop,Performing Arts Venue,Sandwich Place,Gym,Pub,Pizza Place,Pharmacy,Convenience Store,Cupcake Shop,Cuban Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Mediterranean Restaurant,Pizza Place,Middle Eastern Restaurant,Coffee Shop,Deli / Bodega,Fried Chicken Joint,Farmers Market,Escape Room,Ethiopian Restaurant,Event Space
3,Bayview Village,Flower Shop,Gas Station,Trail,Park,Event Space,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Café,Greek Restaurant,Sushi Restaurant,Pub,Liquor Store,Juice Bar,Thai Restaurant


In [44]:
toronto_data= toronto_data.rename(columns = {"Neighbourhood" : "Neighborhood"})
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.6505,-79.5517
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.6437,-79.5767
3,M9P,Etobicoke,Westmount,43.6949,-79.5323
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.6898,-79.5582


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282,4.0,Pharmacy,Park,Baseball Field,Skating Rink,Bank,Grocery Store,Women's Store,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.6505,-79.5517,1.0,Pizza Place,Construction & Landscaping,Chinese Restaurant,Sandwich Place,Print Shop,Tea Room,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.6437,-79.5767,1.0,Pizza Place,Shopping Plaza,Coffee Shop,Beer Store,Liquor Store,Café,Convenience Store,Pharmacy,Field,Fast Food Restaurant
3,M9P,Etobicoke,Westmount,43.6949,-79.5323,1.0,Flea Market,Supermarket,Sandwich Place,Coffee Shop,Discount Store,Chinese Restaurant,Ice Cream Shop,Pizza Place,Field,Fast Food Restaurant
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.6898,-79.5582,1.0,Pharmacy,Bus Line,Sandwich Place,Chinese Restaurant,Supermarket,Beer Store,Gas Station,Bank,Pizza Place,American Restaurant


Finally, let's visualize the resulting clusters

In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 2

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Etobicoke,1.0,Pizza Place,Construction & Landscaping,Chinese Restaurant,Sandwich Place,Print Shop,Tea Room,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
2,Etobicoke,1.0,Pizza Place,Shopping Plaza,Coffee Shop,Beer Store,Liquor Store,Café,Convenience Store,Pharmacy,Field,Fast Food Restaurant
3,Etobicoke,1.0,Flea Market,Supermarket,Sandwich Place,Coffee Shop,Discount Store,Chinese Restaurant,Ice Cream Shop,Pizza Place,Field,Fast Food Restaurant
4,Etobicoke,1.0,Pharmacy,Bus Line,Sandwich Place,Chinese Restaurant,Supermarket,Beer Store,Gas Station,Bank,Pizza Place,American Restaurant
6,Etobicoke,1.0,Grocery Store,Sandwich Place,Pharmacy,Pizza Place,Fast Food Restaurant,Hardware Store,Fried Chicken Joint,Beer Store,Caribbean Restaurant,Liquor Store
7,Etobicoke,1.0,Coffee Shop,Performing Arts Venue,Sandwich Place,Gym,Pub,Pizza Place,Pharmacy,Convenience Store,Cupcake Shop,Cuban Restaurant
8,Etobicoke,1.0,Mediterranean Restaurant,Hotel,Coffee Shop,Swiss Restaurant,Falafel Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Farmers Market
9,Etobicoke,1.0,Bank,Breakfast Spot,Sushi Restaurant,Bakery,Pub,Liquor Store,Coffee Shop,Restaurant,Bar,Smoke Shop
11,Etobicoke,1.0,Burrito Place,Italian Restaurant,Burger Joint,Social Club,Middle Eastern Restaurant,Liquor Store,Sushi Restaurant,Thai Restaurant,Gym,Gym / Fitness Center


#### Cluster 3

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 4

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 5

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,4.0,Pharmacy,Park,Baseball Field,Skating Rink,Bank,Grocery Store,Women's Store,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store
10,Etobicoke,4.0,Park,Home Service,Baseball Field,Construction & Landscaping,Women's Store,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
